# <p style="text-align: center;, font-style: strong;">Partie 2.1 : MNIST with Multiple Layer Perceptron (MLP)</p>


## Dependencies

In [4]:
interp.load.ivy(
  coursier.Dependency(
    module = coursier.Module(coursier.Organization("org.platanios"), coursier.ModuleName("tensorflow_2.12")),
    version = "0.4.1",
    // replace with linux-gpu-x86_64 on linux with nvidia gpu or with darwin-cpu-x86_64 on macOS 
    attributes = coursier.Attributes(coursier.Type(""), coursier.Classifier("darwin-cpu-x86_64"))
  )
)

interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")


In [5]:
import org.platanios.tensorflow.api.ops.{Files, Image => TImage}
import java.nio.file.{ Files => JFiles, Paths, Path }
import java.awt.{Image => JImage}
import java.io.{BufferedInputStream, File, FileInputStream, ByteArrayOutputStream, IOException}
import scala.io.Source
import scala.collection.mutable.ListBuffer
import java.awt.image.BufferedImage
import java.awt.Color
import java.awt.image.BufferedImage
import javax.imageio.ImageIO
import scala.util.Random
import java.util.Base64
import java.io.PrintWriter
import scala.io.Source
import java.io.ByteArrayInputStream
import org.platanios.tensorflow.api.tf
import org.platanios.tensorflow.api.tensors.Tensor
import org.platanios.tensorflow.api.core.Shape
import org.platanios.tensorflow.api.core.client.{Session, FeedMap}
import org.platanios.tensorflow.api.core.types.DataType
import org.platanios.tensorflow.data.image.MNISTLoader
import java.nio.file.{Files, Paths}
import org.platanios.tensorflow.api.core.{Shape, types}
import scala.language.postfixOps
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.core.types.UByte
import org.platanios.tensorflow.api.implicits.helpers.{OutputStructure, OutputToDataType, OutputToShape}
import org.platanios.tensorflow.api.learn.ClipGradientsByGlobalNorm
import org.platanios.tensorflow.api.ops.Output
import org.platanios.tensorflow.data.image.MNISTLoader
import org.platanios.tensorflow.api.learn.layers.{Flatten, Input, Linear, ReLU, SparseSoftmaxCrossEntropy, Mean, ScalarSummary}
import org.platanios.tensorflow.api.learn.{Model, StopCriteria, Configuration}
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.api.learn.hooks.{SummarySaver, StepHookTrigger, CheckpointSaver}
import org.platanios.tensorflow.api.config.TensorBoardConfig
import java.util.Base64
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.core.types.UByte
import org.platanios.tensorflow.api.implicits.helpers.{OutputStructure, OutputToDataType, OutputToShape}
import org.platanios.tensorflow.api.learn.ClipGradientsByGlobalNorm
import org.platanios.tensorflow.api.ops.Output
import org.platanios.tensorflow.data.image.MNISTLoader
import java.nio.file.Paths

import org.platanios.tensorflow.api.ops.{Files, Image => TImage}

import java.nio.file.{ Files => JFiles, Paths, Path }

import java.awt.{Image => JImage}

import java.io.{BufferedInputStream, File, FileInputStream, ByteArrayOutputStream, IOException}

import scala.io.Source

import scala.collection.mutable.ListBuffer

import java.awt.image.BufferedImage

import java.awt.Color

import java.awt.image.BufferedImage

import javax.imageio.ImageIO

import scala.util.Random

import java.util.Base64

import java.io.PrintWriter

import scala.io.Source

import java.io.ByteArrayInputStream

import org.platanios.tensorflow.api.tf

import org.platanios.tensorflow.api.tensors.Tensor

import org.platanios.tensorflow.api.core.Shape

import org.platanios.tensorflow.api.core.client.{Session, FeedMap}

import org.platanios.tensorflow.api.core.types.DataType

import org.platanios.tensorflow.data.image.MNISTLoader

import java.nio.file.{Files, Paths}

import org.platanios.tensorflow.api.core.{Shape, types

## Display MNIST Dataset

In [3]:
def displayNumberMNIST(nb: Int) {
    val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))
    val images = dataset.trainImages
    val imagesToDisplay = images.slice(0 :: nb, ::, ::)
    var s = true
    for (index <- 0 until nb) {
        val png = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(imagesToDisplay(index).reshape(Shape(28, 28, 1)))))
        Image(png.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    }
}
displayNumberMNIST(20)

2019-07-25 08:56:34.603 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/train-images-idx3-ubyte.gz'.
2019-07-25 08:56:36.008 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/train-labels-idx1-ubyte.gz'.
2019-07-25 08:56:36.013 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/t10k-images-idx3-ubyte.gz'.
2019-07-25 08:56:36.077 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/t10k-labels-idx1-ubyte.gz'.
2019-07-25 08:56:36.080 [scala-interpreter-1] INFO  MNIST Data Loader - Finished loading the MNIST dataset.


defined function displayNumberMNIST

In [6]:
val dataSet = MNISTLoader.load(Paths.get("../resources/dataset"))
val trainImages = tf.data.datasetFromTensorSlices(dataSet.trainImages).map(_.toFloat)
val trainLabels = tf.data.datasetFromTensorSlices(dataSet.trainLabels).map(_.toLong)
val testImages = tf.data.datasetFromTensorSlices(dataSet.testImages).map(_.toFloat)
val testLabels = tf.data.datasetFromTensorSlices(dataSet.testLabels).map(_.toLong)
val trainData =
  trainImages.zip(trainLabels)
      .repeat()
      .shuffle(10000)
      .batch(256)
      .prefetch(10)
val evalTrainData = trainImages.zip(trainLabels).batch(1000).prefetch(10)
val evalTestData = testImages.zip(testLabels).batch(1000).prefetch(10)


val input = tf.learn.Input(FLOAT32, Shape(-1, dataSet.trainImages.shape(1), dataSet.trainImages.shape(2)))
val trainInput = tf.learn.Input(INT64, Shape(-1))
val layer = tf.learn.Flatten[Float]("Input/Flatten") >>
    tf.learn.Linear[Float]("Layer_0/Linear", 128) >> tf.learn.ReLU[Float]("Layer_0/ReLU", 0.1f) >>
    tf.learn.Linear[Float]("Layer_1/Linear", 64) >> tf.learn.ReLU[Float]("Layer_1/ReLU", 0.1f) >>
    tf.learn.Linear[Float]("Layer_2/Linear", 32) >> tf.learn.ReLU[Float]("Layer_2/ReLU", 0.1f) >>
    tf.learn.Linear[Float]("OutputLayer/Linear", 10)
val loss = tf.learn.SparseSoftmaxCrossEntropy[Float, Long, Float]("Loss/CrossEntropy") >>
    tf.learn.Mean[Float]("Loss/Mean") >>
    tf.learn.ScalarSummary[Float]("Loss/Summary", "Loss")
val optimizer = tf.train.YellowFin()

val model = tf.learn.Model.simpleSupervised(
  input = input,
  trainInput = trainInput,
  layer = layer,
  loss = loss,
  optimizer = optimizer,
  clipGradients = ClipGradientsByGlobalNorm(5.0f))

val summariesDir = Paths.get("../resources/save")
val accMetric = tf.metrics.MapMetric(
  (v: (Output[Float], (Output[Float], Output[Int]))) => {
    (tf.argmax(v._1, -1, INT64).toFloat, v._2._2.toFloat)
  }, tf.metrics.Accuracy("Accuracy"))
val estimator = tf.learn.InMemoryEstimator(
  model,
  tf.learn.Configuration(Some(summariesDir)),
  tf.learn.StopCriteria(maxSteps = Some(100000)),
  Set(
    tf.learn.LossLogger(trigger = tf.learn.StepHookTrigger(100)),
    tf.learn.Evaluator(
      log = true, datasets = Seq(("Train", () => evalTrainData), ("Test", () => evalTestData)),
      metrics = Seq(accMetric), trigger = tf.learn.StepHookTrigger(1000), name = "Evaluator"),
    tf.learn.StepRateLogger(log = false, summaryDir = summariesDir, trigger = tf.learn.StepHookTrigger(100)),
    tf.learn.SummarySaver(summariesDir, tf.learn.StepHookTrigger(100)),
    tf.learn.CheckpointSaver(summariesDir, tf.learn.StepHookTrigger(1000))),
  tensorBoardConfig = tf.learn.TensorBoardConfig(summariesDir, reloadInterval = 1))
estimator.train(() => trainData, tf.learn.StopCriteria(maxSteps = Some(10000)))

def accuracy(images: Tensor[UByte], labels: Tensor[UByte]): Float = {
  val predictions = estimator.infer(() => images.toFloat)
  predictions
      .argmax(1).toUByte
      .equal(labels).toFloat
      .mean().scalar
}


2019-07-25 08:57:53.189 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/train-images-idx3-ubyte.gz'.
2019-07-25 08:57:53.552 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/train-labels-idx1-ubyte.gz'.
2019-07-25 08:57:53.554 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/t10k-images-idx3-ubyte.gz'.
2019-07-25 08:57:53.606 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/t10k-labels-idx1-ubyte.gz'.
2019-07-25 08:57:53.608 [scala-interpreter-1] INFO  MNIST Data Loader - Finished loading the MNIST dataset.
2019-07-25 08:57:58.674 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 0.
2019-07-25 08:57:58.676 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/Users/vincentbrule/Desktop/audiScala/WORKSHOP/PARTIE_2/notebooks/../resources/

2019-07-25 08:58:48.466 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    1.456 s) Step:   2600, Loss: 2.6773
2019-07-25 08:58:50.006 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    1.541 s) Step:   2700, Loss: 1.8804
2019-07-25 08:58:51.383 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    1.377 s) Step:   2800, Loss: 2.7786
2019-07-25 08:58:52.735 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    1.352 s) Step:   2900, Loss: 1.5707
2019-07-25 08:58:54.061 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 3000.
2019-07-25 08:58:54.062 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/Users/vincentbrule/Desktop/audiScala/WORKSHOP/PARTIE_2/notebooks/../resources/save/model.ckpt-3000'.
2019-07-25 08:58:54.274 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/Users/vincentbrule/Desktop/audiScala/WORKSHOP/PARTIE_2/notebooks/../resources/save/model.ckpt-3000

2019-07-25 08:59:45.797 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    1.373 s) Step:   5800, Loss: 0.4473
2019-07-25 08:59:47.102 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    1.305 s) Step:   5900, Loss: 0.2930
2019-07-25 08:59:48.374 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 6000.
2019-07-25 08:59:48.375 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/Users/vincentbrule/Desktop/audiScala/WORKSHOP/PARTIE_2/notebooks/../resources/save/model.ckpt-6000'.
2019-07-25 08:59:48.573 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/Users/vincentbrule/Desktop/audiScala/WORKSHOP/PARTIE_2/notebooks/../resources/save/model.ckpt-6000'.
2019-07-25 08:59:48.573 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    1.471 s) Step:   6000, Loss: 0.3259
2019-07-25 08:59:48.576 [scala-interpreter-1] INFO  Variables / Saver - Restoring parameters from '/Users/vincentbrule/D

2019-07-25 09:00:39.667 [scala-interpreter-1] INFO  Learn / Hooks / Checkpoint Saver - Saving checkpoint for step 9000.
2019-07-25 09:00:39.668 [scala-interpreter-1] INFO  Variables / Saver - Saving parameters to '/Users/vincentbrule/Desktop/audiScala/WORKSHOP/PARTIE_2/notebooks/../resources/save/model.ckpt-9000'.
2019-07-25 09:00:39.866 [scala-interpreter-1] INFO  Variables / Saver - Saved parameters to '/Users/vincentbrule/Desktop/audiScala/WORKSHOP/PARTIE_2/notebooks/../resources/save/model.ckpt-9000'.
2019-07-25 09:00:39.867 [scala-interpreter-1] INFO  Learn / Hooks / Loss Logger - (    1.427 s) Step:   9000, Loss: 0.3805
2019-07-25 09:00:39.869 [scala-interpreter-1] INFO  Variables / Saver - Restoring parameters from '/Users/vincentbrule/Desktop/audiScala/WORKSHOP/PARTIE_2/notebooks/../resources/save/model.ckpt-9000'.
2019-07-25 09:00:40.188 [scala-interpreter-1] INFO  Learn / Hooks / Evaluation - Step 9000 Evaluator:
2019-07-25 09:00:40.189 [scala-interpreter-1] INFO  Learn / Hoo

dataSet: org.platanios.tensorflow.data.image.MNISTDataset = MNISTDataset(
  MNIST,
  Tensor[UByte, [60000, 28, 28]],
  Tensor[UByte, [60000]],
  Tensor[UByte, [10000, 28, 28]],
  Tensor[UByte, [10000]]
)
trainImages: ops.data.Dataset[Output[Float]] = Dataset[TensorSlicesDataset/Map]
trainLabels: ops.data.Dataset[Output[Long]] = Dataset[TensorSlicesDataset/Map]
testImages: ops.data.Dataset[Output[Float]] = Dataset[TensorSlicesDataset/Map]
testLabels: ops.data.Dataset[Output[Long]] = Dataset[TensorSlicesDataset/Map]
trainData: ops.data.Dataset[(Output[Float], Output[Long])] = Dataset[TensorSlicesDataset/Map/Zip/Repeat/Shuffle/Batch/Prefetch]
evalTrainData: ops.data.Dataset[(Output[Float], Output[Long])] = Dataset[TensorSlicesDataset/Map/Zip/Batch/Prefetch]
evalTestData: ops.data.Dataset[(Output[Float], Output[Long])] = Dataset[TensorSlicesDataset/Map/Zip/Batch/Prefetch]
input: Input[Output[Float]] = org.platanios.tensorflow.api.learn.layers.Input@b6f8926
trainInput: Input[Output[Long]] =

In [8]:
def accuracy(images: Tensor[UByte], labels: Tensor[UByte]): Float = {
    val predictions = estimator.infer(() => images.toFloat)
    predictions
      .argmax(1).toUByte
      .equal(labels).toFloat
      .mean().scalar
}

println(s"Train accuracy = ${accuracy(dataSet.trainImages, dataSet.trainLabels)}")
println(s"Test accuracy = ${accuracy(dataSet.testImages, dataSet.testLabels)}")

Train accuracy = 0.95735
Test accuracy = 0.941


defined function accuracy

In [7]:
/*def transformImagesForPrediction(imagefilePath: String, numChannel: Int, s: Session = Session()): Tensor[UByte] = {
    val tensorImageOuts = TImage.decodePng(Files.readFile(imagefilePath), numChannel)
    s.run(fetches = tf.reshape(tensorImageOuts, Shape(1,28,28)))
}

def runPrediction(imagePath: String): String = {
    val imageTest = transformImagesForPrediction(imagePath, 1)
    val imageToDisplay = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(tf.reshape(imageTest, Shape(28, 28, 1)))))
    Image(imageToDisplay.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    val predictions = estimator.infer(() => imageTest.toFloat)
    s"Le chiffre est : ${predictions.argmax(1).toInt.scalar}"
}*/
dataSet.trainImages.shape

res6: Shape = [60000, 28, 28]